In [1]:
# import libraries

'''
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer
'''

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

# These are all the imports needed for the assignment
%matplotlib inline

# Import nltk package (Natural Language Toolkit)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')

# scikit-learn imports
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [2]:
import pyspark

In [3]:
import dask.dataframe as dd

In [4]:
# Download the NLTK English tokenizer and the stopwords of all languages
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sukan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sukan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

client = boto3.client(
    's3',
    aws_access_key_id = 'sukiK@2002',
    aws_secret_access_key = 'sukik',
    region_name = 'ap-west-1'
)

In [5]:
files = ['amazon_reviews_us_Electronics_v1_00.tsv', \
         'amazon_reviews_us_Gift_Card_v1_00.tsv', \
         'amazon_reviews_us_Major_Appliances_v1_00.tsv', \
         'amazon_reviews_us_Office_Products_v1_00.tsv', \
         'amazon_reviews_us_Shoes_v1_00.tsv', \
         'amazon_reviews_us_Toys_v1_00.tsv', \
         'amazon_reviews_us_Watches_v1_00.tsv']

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [7]:
columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', \
           'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']
elec_df = pd.read_csv('fars_data/' + files[0], names = columns, sep = '\t').iloc[1:,:]
elec_df = elec_df.sample(n = 20000)

columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']
len(columns)

one_file = pd.read_csv('fars_data/' + files[0], names = columns, sep = '\t').iloc[1:,:]
one_file.head()

one_file.shape[0]+1

In [8]:
one_file = elec_df

In [9]:
def df_sampling(df):
    # Since we know that there are more unverified than verified --> we sample based on that
    
    # Since there are no data values in 'verified_purchase' columns that deviate from 'Y' or 'N' we proceed
    verified_count_df = df[df['verified_purchase'] == 'Y']
    unverified_count_df = df[df['verified_purchase'] == 'N']
    
    print("Number of verified purchases:", len(verified_count_df))
    print("Number of unverified purchases:", len(unverified_count_df))
    
    sample_len = len(unverified_count_df)
    
    verified_sample_df = verified_count_df.sample(n = sample_len)
    unified_df = pd.concat([unverified_count_df, verified_sample_df])
    
    print("Number of verified purchases (balanced dataset):", len(unified_df[unified_df['verified_purchase'] == 'Y']))
    print("Number of unverified purchases (balanced dataset):", len(unified_df[unified_df['verified_purchase'] == 'N']))
    
    return unified_df

#test.rename(columns=test.iloc[0])
training_data = pd.DataFrame() #initialize an empty dataframe
testing_data = pd.DataFrame()

one_file = pd.read_csv(data_location + files[1], names = columns, sep = '\t').iloc[1:,:]

In [10]:
one_file.shape

(20000, 15)

In [11]:
balanced_elec = df_sampling(one_file)
display(balanced_elec.head())

Number of verified purchases: 18031
Number of unverified purchases: 1969
Number of verified purchases (balanced dataset): 1969
Number of unverified purchases (balanced dataset): 1969


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
931083,US,16628610,R1WV32MG7A6ILD,B0000DHVOO,355100210,Crosley Solo Radio,Electronics,5,2,2,N,N,It's got fantastic sound quality and construction,"I've had this radio for several years. It's got fantastic sound quality and construction. I lost the antenna at some point during one of my moves, but it still works great at tuning radio stations without it. Cable hooks up to my ipod/phone/computer sound jack.",2014-11-25
1188044,US,44092743,R1PUA20HJCEKB4,B003XM9774,508552334,HDMI-High-Speed,Electronics,5,0,0,N,N,5 Star HDMI,Perfect cable that will not break. I am about to buy another one for my upstairs TV.,2014-08-17
1128381,US,42999527,R25EF5EHDJRVJ7,B00MX37P64,83189676,"ALL NEW 2015®[PREMIUM] 6FT AUX HQ-CONCRETE Series [Liquid Black] - 3.5mm [6 Foot] for Apple iPhone, Andriod, Tablet, iPad, iPod, MP3 Player, Samsung Galaxy Note (For All 3.5mm Enabled devices) Male To Male - Aux Auxiliary Stereo Car Jack Cable",Electronics,5,1,1,N,N,Affordable premium cable,Bought this to plug into my Bose speaker and the sound quality is as promised. The cord material seems very strong. It's also a pretty good looking cord.,2014-09-09
490933,US,1010350,R2PRXHO678DLT5,B00SWDHUXW,25324900,Status Audio HD One Headphones - Noise isolating. Matte finish. Foldable. 2 cables. Mic.,Electronics,5,0,0,N,N,Awesome!!!,The best headphones for the money period!!!!,2015-03-26
1081221,US,26040979,R2NLQWSD9ORV57,B0051GOC3Y,214887992,Sony NWZ-B162F Flash MP3 Player (2GB) - Green,Electronics,5,0,0,N,N,Five Stars,"prodcuto excellent, very happy and satisfied with the purchase",2014-09-27


In [12]:
def convert_to_int(x):
    return int(float(x))

In [13]:
#convert all mixed datatypes --> string objects (unable to convert to int)
one_file['customer_id'] = one_file['customer_id'].apply(int) #1
one_file['product_parent'] = one_file['product_parent'].apply(int) #4
one_file['star_rating'] = one_file['star_rating'].apply(int) #7
one_file['helpful_votes'] = one_file['helpful_votes'].apply(int) #8
one_file['total_votes'] = one_file['total_votes'].apply(int) #9

one_file.isnull().sum().sort_values(ascending=False)

one_file.dropna(inplace=True) #drop all of the missing values
one_file.isnull().sum().sort_values(ascending=False)

In [14]:
#there may be tabs in the review that is what is leading to the excluded cases when converting to df

'''
train_data['verified_purchase'].value_counts()
test_data['verified_purchase'].value_counts()
'''

one_file.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
884958,US,18117238,R2HAH32AF7863Q,B00I6N0TH2,627742809,House of Marley Chant Drift Earbud Headphones,Electronics,5,0,0,N,Y,Great value and look!,Great earbuds with a great sound. You can not buy a better looking earbud with this quality of sound for less.,2014-12-10
612788,US,26686997,R3LHRS6FP2GZR9,B003ZKBWAY,198367226,Nintendo 64 AC Adapter (Power Supply),Electronics,4,0,0,N,Y,Works great.,"This is an aftermarket power supply, but fits fine and powers the system with no issues. One thing that would be nice is a longer cord.",2015-02-21
995823,US,24454643,RP9B5AN2JTPB2,B00GZC35YK,303943466,"Anker Classic Portable Wireless Bluetooth Speaker, Powerful Sound with Enhanced Bass, 20 Hour Battery Life, and Built-in Mic, works with iPhone, iPad,Nexus, Laptops and More",Electronics,5,0,0,N,Y,Worth every penny,"Use it everyday. I drive for a living and the vehicle provided to me doesn't come with a radio. So this is my radio, blue toothed to my I-phone. Some days I use it for 12hrs+ on the same battery life and it doesn't die on me, which is nice. I also like how it is shaped in a cube so it doesn't roll around on the dash while I am driving. Even though the music is provided through my phone it doesn't drain battery life on my phone very much. The only thing I would change is to add another button to the device to change to next and previous songs. It gets annoying if I am already using my phone for something and then have to change/close apps to my music app just to change songs",2014-10-31
317978,US,52413157,R17QMT4MWLMRDP,B004YEBK66,32110033,"Mediabridge - 3.5mm Male To (2) RCA Stereo Audio Cable- Step Down Design accommodates iPhone, iTouch, SmartPhones & MP3 cases",Electronics,5,0,0,N,Y,Pricey but worth it,I ususally buy Mediabridge cables. They are not the least expensive but are consistently high quality. These did not disappoint!,2015-05-23
963613,US,48185535,RP18JKON8WQQN,B004LZ5XMU,596015880,Accell D080B-026K Power Squid Outlet Multiplier with 5 Power Outlets,Electronics,5,0,0,N,Y,Five Stars,great,2014-11-14


#to check te datatypes within each column

print(train_data.applymap(type))
data_types = train_data.applymap(type)

In [15]:
#the number of customers that gave multiple reviews
#may be valueable to investigate
one_file.groupby('customer_id').count()['marketplace'].sort_values(ascending = False).value_counts()

1    19669
2      153
3        5
4        1
6        1
Name: marketplace, dtype: int64

In [16]:
one_file['star_rating'].value_counts()

5    12230
4     3101
1     2146
3     1445
2     1078
Name: star_rating, dtype: int64

Data Cleaning

In [17]:
# Import re for text cleaning purposes
import re

import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sukan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sukan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
def df_cleaning(df):
    # Drop rows with na values
    df.dropna(inplace = True)
    
    df['new_review_body'] = df['review_body'].copy()
    
    # Remove unwanted formatting characters
    format_strs = dict.fromkeys(['<br /><br />', '&#34', 'br', '&quot', '<br />'], ' ')
    for key in format_strs:
        df['new_review_body'] = df['new_review_body'].apply(lambda review: review.replace(key, format_strs[key]))
    # removing quotes produces smthg like this --> 'The product has great ;sound; --> we must remove punctuation

    
    # Case normalization (lower case)
    df['new_review_body'] = df['new_review_body'].str.lower()
    
    remove_dict = {"0": "", "1": "", "2": "", "3": "", "4": "", "5": "", "6": "", "7": "", "8": "", "9": "",
                   "(": "", ")":""}
    for key, val in remove_dict.items():
        df['new_review_body'] = df['new_review_body'].apply(
            lambda x: x.replace(key, val))
        
    # Remove stopwords
    stop_lst = stopwords.words('english')
    #stop_lst += (["can't", "i'm" "i'd", "i've", "i'll", "that's", "there's", "they're"])
    # ****Do we not have to take stopwords out BEFORE removing punctuation? Otherwise words with punct like “cant” remains there
    df['new_review_body'] = df['new_review_body'].apply(lambda text_body: " ".join([word for word in text_body.split() if word not in (stop_lst)]))
    
    # Removing Unicode Chars (punctuation, URL, @)
    df['new_review_body'] = df['new_review_body'].apply(lambda rev: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", rev))
    
    # Lemmatization
    word_lemmatizer = WordNetLemmatizer()
    df['new_review_body'] = df['new_review_body'].apply(lambda txt: " ".join([(word_lemmatizer.lemmatize(word)) for word in txt.split()]))
    
    return df

cleaned = df_cleaning(one_file)

In [21]:
cleaned2 = df_cleaning(one_file)

In [34]:
cleaned2.get(['review_body', 'new_review_body']).head()

,review_body,new_review_body
884958,Great earbuds with a great sound. You can not buy a better looking earbud with this quality of sound for less.,great earbuds great sound buy better looking earbud quality sound le
612788,"This is an aftermarket power supply, but fits fine and powers the system with no issues. One thing that would be nice is a longer cord.",aftermarket power supply fit fine power system issue one thing would nice longer cord
995823,"Use it everyday. I drive for a living and the vehicle provided to me doesn't come with a radio. So this is my radio, blue toothed to my I-phone. Some days I use it for 12hrs+ on the same battery life and it doesn't die on me, which is nice. I also like how it is shaped in a cube so it doesn't roll around on the dash while I am driving. Even though the music is provided through my phone it doesn't drain battery life on my phone very much. The only thing I would change is to add another button to the device to change to next and previous songs. It gets annoying if I am already using my phone for something and then have to change/close apps to my music app just to change songs",use everyday drive living vehicle provided come radio radio blue toothed iphone day use hr battery life die me nice also like shaped cube roll around dash driving even though music provided phone drain battery life phone much thing would change add another button device change next previous song get annoying already using phone something changeclose apps music app change song
317978,I ususally buy Mediabridge cables. They are not the least expensive but are consistently high quality. These did not disappoint!,ususally buy medium idge cable least expensive consistently high quality disappoint
963613,great,great


In [35]:
cleaned2.iloc[0].get('review_body')

'Great earbuds with a great sound. You can not buy a better looking earbud with this quality of sound for less.'

In [36]:
get_sentiment_scores(cleaned2.iloc[0].get('review_body')) #without lemmitization

{'neg': 0.084, 'neu': 0.629, 'pos': 0.287, 'compound': 0.7779}

In [38]:
get_sentiment_scores(cleaned2.iloc[0].get('new_review_body')) #with lemmitization

{'neg': 0.0, 'neu': 0.419, 'pos': 0.581, 'compound': 0.9022}

In [39]:
one_file = cleaned2

### Vader Sentiment Analysis

In [58]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def get_sentiment_scores(review):
    """
    create new dataframe with just the proportions for each review
    four columns
    neg_prop, pos_prop, neu_prop, compound_prop and will contain these values
    obtained from the vator sentiment algorithm
    """
    snt = analyser.polarity_scores(review)
    #print(f"{sentence} {str(snt)}")
    return snt

In [59]:
one_file['rev_dict'] = one_file['new_review_body'].apply(get_sentiment_scores)

In [60]:
def get_neg(review_dict):
    return review_dict['neg']

def get_neu(review_dict):
    return review_dict['neu']

def get_pos(review_dict):
    return review_dict['pos']

def get_compound(review_dict):
    return review_dict['compound']

def only_compound(x):
    dct = get_sentiment_scores(x)
    return dct['compound']

In [61]:
#get neg prop
one_file['neg_prop'] = one_file['rev_dict'].apply(get_neg)
#get neu prop
one_file['neu_prop'] = one_file['rev_dict'].apply(get_neu)
#get pos prop
one_file['pos_prop'] = one_file['rev_dict'].apply(get_pos)
#get compound prop
one_file['compound_prop'] = one_file['rev_dict'].apply(get_compound)

In [62]:
#save the dataframe as a csv file
one_file.to_csv('electronics_data_cleaned.csv')

### Sentiment Analysis

#df is all data other than rating 3
df = one_file[one_file['star_rating'] != 3]

df['star_rating'].value_counts()

df['positively_rated'] = np.where(df['star_rating'] > 3, 1, 0)

df['positively_rated'].value_counts()

sns.countplot(df['positively_rated'])

##### On all of the data

train_data, test_data = np.split(one_file.sample(frac=1, random_state=1729), [int(0.7 * len(one_file))])
print(train_data.shape, test_data.shape)

X_train = train_data['review_body']
X_train.iloc[0] #to_frame()

Y_train = train_data['star_rating']

X_test = test_data['review_body']
Y_test = test_data['star_rating']

#### TF-IDF - to get predictions of what star rating will be based on text of review

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer().fit(X_train)

len(vect.get_feature_names()) #unique words

X_train_vect = vect.transform(X_train)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vect, Y_train)

predictions = model.predict(vect.transform(X_test))

len(predictions)

###### Confusion Matrix - shows probabilities for whether cat data is predicted correctly

In [63]:
def plot_confusion_matrix(cm, target_names,
                          fname, epoch,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True, target=None):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools
    plt.style.use('default')

    # # only true if it weren't normalized:
    # accuracy = np.trace(cm) / float(np.sum(cm))
    # misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm[np.isnan(cm)] = 0.0

    fig = plt.figure(figsize=(5, 4))
    ax = plt.axes()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    if target == "rule-based":
        plt.title(title + ' for rule-based PF')
    else:
        plt.title(title + ' for MLPF at epoch ' + str(epoch))

    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlim(-1, len(target_names))
    plt.ylim(-1, len(target_names))
    plt.xlabel('Predicted label')
    # plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    plt.savefig(fname + '.png')
    plt.savefig(fname + '.pdf')
    #plt.close(fig)

    return fig, ax

### SVM Notes

- **Bag of Words (BoW)**
        - BoW encodes an input sentence as the frequency of each word in the sentence. 
        - In this approach, all words contribute equally to the feature vectors.
- **Term Frequency - Inverse Document Frequency (TF-IDF)**
        - TF-IDF is a measure of how important each term is to a specific document, as compared to an overall corpus. 
        - TF-IDF encodes each word as its frequency in the document of interest, divided by a measure of how common the word is across all documents (the corpus).
        - Using this approach, each word contributes differently to the feature vectors.
        - The assumption behind using TF-IDF is that words that appear commonly everywhere are not that informative about what is specifically interesting about a document of interest, so it is tuned to representing a document in terms of the words it uses that are different from other documents. 

- To compare those 2 methods, we will first apply them on the same dataset to analyse sentiment (how positive or negative a text is). In order to make the comparison fair, an **SVM (support vector machine)** classifier will be used to classify positive reviews and negative reviews.

- SVM is a simple yet powerful and interpretable linear model. To use it as a classifier, we need to have at least 2 splits of the data: training data and test data. The training data is used to tune the weight parameters in the SVM to learn an optimal way to classify the training data. We can then test this trained SVM classifier on the test data, to see how well it works on data that the classifier has not seen before. 

We will now create a CountVectorizer object to transform the text data into vectors with numerical values.

To do so, we will initialize a CountVectorizer object, and name it as vectorizer.

4 arguments to initialize a CountVectorizer:

* analyzer: 'word'
 Specify to analyze data from word-level.
 
* max_features: 2000
 Set a max number of unique words.
 
* tokenizer: word_tokenize
 Set to tokenize the text data by using the word_tokenizer from NLTK .
 
* stop_words: stopwords.words('english')
 Set to remove all stopwords in English. We do this since they generally don't provide useful discriminative information.

## KNN

In [64]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [65]:
one_file.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date', 'new_review_body',
       'rev_dict', 'neg_prop', 'neu_prop', 'pos_prop', 'compound_prop',
       'product_id_convert', 'product_category_convert', 'prod_title_comp',
       'rev_title_comp'],
      dtype='object')

In [66]:
def convert_to_id(x):
    alphabet = {'a': 1, 'c': 3, 'b': 2, 'e': 5, 'd': 4, 'g': 7, 'f': 6, 'i': 9, 'h': 8, 'k': 11, 'j': 10, 'm': 13, 'l': 12, 'o': 15, 'n': 14, 'q': 17, 'p': 16, 's': 19, 'r': 18, 'u': 21, 't': 20, 'w': 23, 'v': 22, 'y': 25, 'x': 24, 'z': 26}
    out = ''
    for i in x:
        if i.lower() in alphabet.keys():
            out += str(alphabet[i.lower()])
        else:
            out += i
    return out

In [67]:
one_file['product_id_convert'] = one_file.get("product_id").apply(convert_to_id)

In [68]:
one_file['product_category_convert'] = one_file.get("product_category").apply(convert_to_id)

In [69]:
one_file['prod_title_comp'] = one_file.get("product_title").apply(only_compound)

In [70]:
one_file['rev_title_comp'] = one_file.get("review_headline").apply(only_compound)

In [71]:
one_file.iloc[0].to_frame()

,884958
marketplace,US
customer_id,18117238
review_id,R2HAH32AF7863Q
product_id,B00I6N0TH2
product_parent,627742809
product_title,House of Marley Chant Drift Earbud Headphones
product_category,Electronics
star_rating,5
helpful_votes,0
total_votes,0


find percentages for the body -- text and display those in a column -- act as labels for the percent positivity of the review body and header themselves

In [181]:
imp_col = one_file[['verified_purchase', 'customer_id', 'product_id_convert', 'prod_title_comp', 'product_category_convert', 'star_rating', 'helpful_votes', 'total_votes', 'rev_title_comp', 'neg_prop', 'neu_prop', 'pos_prop']]

In [182]:
imp_col.dtypes

verified_purchase            object
customer_id                   int64
product_id_convert           object
prod_title_comp             float64
product_category_convert     object
star_rating                   int64
helpful_votes                 int64
total_votes                   int64
rev_title_comp              float64
neg_prop                    float64
neu_prop                    float64
pos_prop                    float64
dtype: object

In [183]:
#need to replace review_body and review_headline with percentages for sentiment -- easier to use for classification
#vine needs to be converted to a yes/no binary column similar to how positively rated was determined
#use original dataset with rating 3 available

#X = imp_col.iloc[:, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values
X = imp_col.iloc[:, [3, 5, 6, 7, 8, 9, 10, 11]].values #only taking in the categories that will be used as a dataframe
y = imp_col.iloc[:, 0].values

"""
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,1] = le.fit_transform(X[:,1])
"""

'\nfrom sklearn.preprocessing import LabelEncoder\nle = LabelEncoder()\nX[:,1] = le.fit_transform(X[:,1])\n'

In [184]:
X

array([[0.    , 5.    , 0.    , ..., 0.    , 0.419 , 0.581 ],
       [0.    , 4.    , 0.    , ..., 0.    , 0.608 , 0.392 ],
       [0.4215, 5.    , 0.    , ..., 0.096 , 0.827 , 0.077 ],
       ...,
       [0.    , 5.    , 1.    , ..., 0.    , 0.531 , 0.469 ],
       [0.    , 5.    , 0.    , ..., 0.    , 0.612 , 0.388 ],
       [0.    , 1.    , 7.    , ..., 0.111 , 0.659 , 0.23  ]])

#Since our dataset containing character variables we have to encode it using LabelEncoder

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [185]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [186]:
# Next, we are doing feature scaling to the training and test set of independent variables for reducing the size to smaller values
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [187]:
from sklearn.neighbors import KNeighborsClassifier

#we are using 
#5 neighborhood points are required for classifying a given point -- distance metric is using the minkonowski equation
knn_classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [188]:
y_pred = knn_classifier.predict(X_test)

In [189]:
#We can evaluate our model using the confusion matrix and accuracy score by comparing the predicted and actual test values

from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test,y_pred)

In [190]:
print(cm)

[[  15  380]
 [  34 3571]]


In [191]:
print(ac)

0.8965


In [192]:
#Can see the model performance and add more features accordingly -- 
#would be good if the performance is greater than 85%

Test on a product review

In [207]:
review_test = "this is a good review"
product_title = "Sony Headphones"
review_title = 'Love the product!'
#product_category = "Electronics"
star_rating = 5
helpful_votes = 0
total_votes = 0

In [208]:
test = pd.DataFrame()
test['review_body'] = np.array([review_test])
test

,review_body
0,this is a good review


In [209]:
out = df_cleaning(test)
out

,review_body,new_review_body
0,this is a good review,good review


In [210]:
def get_sentiment_proportions(review):
    """
    create new dataframe with just the proportions for each review
    four columns
    neg_prop, pos_prop, neu_prop, compound_prop and will contain these values
    obtained from the vator sentiment algorithm
    """
    snt = analyser.polarity_scores(review)
    #print(f"{sentence} {str(snt)}")
    neg = snt['neg']
    neu = snt['neu']
    pos = snt['pos']
    #compound = snt['compound']
    return neg, neu, pos

neg, neu, pos = get_sentiment_proportions(out.get("new_review_body").iloc[0])

product_category = convert_to_id(product_category)
product_title = only_compound(product_title)
rev_title = only_compound(review_title)

Predicted: 'verified_purchase'

User Input: 'prod_title_comp', 'product_category_convert', 'star_rating', 'helpful_votes', 'total_votes', 'neg_prop', 'neu_prop', 'pos_prop'
- 8 fields

In [211]:
rev_input_test = np.array([[product_title, star_rating, helpful_votes, total_votes, rev_title, neg, neu, pos]])
rev_input_test

array([[0.    , 5.    , 0.    , 0.    , 0.6696, 0.    , 0.256 , 0.744 ]])

In [212]:
prediction, probabilities = knn_classifier.predict(rev_input_test), knn_classifier.predict_proba(rev_input_test)[0]

In [213]:
prediction

array(['Y'], dtype=object)

In [214]:
probabilities

array([0.2, 0.8])

classifier?

In [215]:
def interpret_prediction(review, pred, proba):
    proba = [round(proba[0], 3), round(proba[1], 3)]
    if prediction[0] == 'Y':
        print(f'"{review}" is predicted to be a VERIFIED review, with {proba[1]*100}% probability of being VERIFIED and {proba[0]*100}% probability of being UNVERIFIED')
    if prediction[0] == 'N':
        print(f'"{review}" is predicted to be an UNVERIFIED review, with {proba[0]*100}% probability of being UNVERIFIED and {proba[1]*100}% probability of being VERIFIED')
        
interpret_prediction(review_test, prediction, probabilities)

"this is a good review" is predicted to be a VERIFIED review, with 80.0% probability of being VERIFIED and 20.0% probability of being UNVERIFIED


In [216]:
from joblib import dump, load

In [218]:
knn_classifier

KNeighborsClassifier()

In [221]:
name = 'knn_working_model.joblib'
path = 'KNNModelFiles/'
dump(knn_classifier, path+name)

['KNNModelFiles/knn_working_model.joblib']

In [222]:
knn_classifier = load(path+name)